<a href="https://colab.research.google.com/github/AbhiJeet70/GraphPoisoningCodes/blob/main/DPGBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install torch-geometric
!pip install ogb
!pip install matplotlib

import torch
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.nn import GCNConv, SAGEConv, GATConv
from torch_geometric.datasets import Planetoid, Flickr
from ogb.nodeproppred import PygNodePropPredDataset
from sklearn.metrics import accuracy_score
import numpy as np
import random
from torch_geometric.utils import subgraph
import pandas as pd
import matplotlib.pyplot as plt

# Set random seeds for reproducibility
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

set_seed()

# Load datasets
def load_dataset(name):
    if name in ['Cora', 'PubMed', 'CiteSeer']:
        return Planetoid(root=f'/tmp/{name}', name=name)
    elif name == 'ogbn-arxiv':
        return PygNodePropPredDataset(name='ogbn-arxiv')
    elif name == 'Flickr':
        return Flickr(root='/tmp/Flickr')
    else:
        raise ValueError(f"Unknown dataset: {name}")

# Define GCN Model
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Define GraphSAGE Model
class GraphSAGE(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(input_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Define GAT Model
class GAT(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GAT, self).__init__()
        self.conv1 = GATConv(input_dim, hidden_dim, heads=8, dropout=0.6)
        self.conv2 = GATConv(hidden_dim * 8, output_dim, heads=1, concat=False, dropout=0.6)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

# Define Trigger Generator (2-layer MLP)
class TriggerGenerator(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(TriggerGenerator, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, input_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

# Define OOD Detector (Discriminator)
class OODDetector(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(OODDetector, self).__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return torch.sigmoid(self.fc2(x))

# Training function
def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward(retain_graph=True)
    optimizer.step()
    return loss.item()

# Test function
def test(model, data):
    model.eval()
    logits = model(data)
    preds = logits.argmax(dim=1)
    accs = []
    for mask in [data.train_mask, data.val_mask, data.test_mask]:
        acc = accuracy_score(data.y[mask].cpu(), preds[mask].cpu())
        accs.append(acc)
    return accs

# Backdoor Attack with Trigger Generator (Adversarial Training with OOD Detector)
def add_backdoor(data, trigger_generator, ood_detector, target_class, num_poisoned, adv_epochs=50):
    poisoned_indices = random.sample(range(data.num_nodes), num_poisoned)
    trigger_optimizer = optim.Adam(trigger_generator.parameters(), lr=0.01)
    ood_optimizer = optim.Adam(ood_detector.parameters(), lr=0.01)

    # Adversarial training loop
    for epoch in range(adv_epochs):
        # Train OOD detector to distinguish between real and generated triggers
        ood_optimizer.zero_grad()
        real_samples = data.x[poisoned_indices]
        fake_samples = trigger_generator(real_samples)
        real_labels = torch.ones(real_samples.size(0), 1)
        fake_labels = torch.zeros(fake_samples.size(0), 1)

        ood_real_loss = F.binary_cross_entropy(ood_detector(real_samples), real_labels)
        ood_fake_loss = F.binary_cross_entropy(ood_detector(fake_samples), fake_labels)
        ood_loss = ood_real_loss + ood_fake_loss
        ood_loss.backward()
        ood_optimizer.step()

        # Train trigger generator to fool the OOD detector
        trigger_optimizer.zero_grad()
        fake_samples = trigger_generator(real_samples)
        fool_labels = torch.ones(fake_samples.size(0), 1)
        generator_loss = F.binary_cross_entropy(ood_detector(fake_samples), fool_labels)
        generator_loss.backward()
        trigger_optimizer.step()

        if epoch % 10 == 0:
            print(f"Adversarial Epoch {epoch}, OOD Loss: {ood_loss.item():.4f}, Generator Loss: {generator_loss.item():.4f}")

    # Apply the generated trigger to the poisoned nodes
    trigger_node_features = trigger_generator(data.x[poisoned_indices])
    data.x[poisoned_indices] = trigger_node_features
    data.y[poisoned_indices] = target_class
    return data, poisoned_indices

# Outlier Detection (using the trained OOD detector)
def apply_outlier_detection(data, ood_detector, threshold=0.5):
    # Use the OOD detector to identify outliers
    with torch.no_grad():
        ood_scores = ood_detector(data.x)
    mask = ood_scores <= threshold

    # Apply mask to nodes and adjust edges accordingly
    data.x = data.x[mask.squeeze()]
    data.y = data.y[mask.squeeze()]
    data.edge_index, _ = subgraph(mask.squeeze(), data.edge_index, relabel_nodes=True, num_nodes=data.num_nodes)

    # Update the train, val, and test masks accordingly
    if hasattr(data, 'train_mask'):
        data.train_mask = data.train_mask[mask.squeeze()]
    if hasattr(data, 'val_mask'):
        data.val_mask = data.val_mask[mask.squeeze()]
    if hasattr(data, 'test_mask'):
        data.test_mask = data.test_mask[mask.squeeze()]

    # Update poisoned indices to reflect only nodes that are still in the dataset
    mask_indices = torch.nonzero(mask.squeeze(), as_tuple=True)[0]
    return data, mask_indices

# Main experiment loop
def run_experiment():
    # Datasets to evaluate
    datasets = ['Cora', 'PubMed', 'Flickr', 'ogbn-arxiv']
    attack_budgets = {'Cora': 10, 'PubMed': 40, 'Flickr': 160, 'ogbn-arxiv': 565}
    trigger_size = 3  # Trigger size limit

    # Hyperparameters
    hidden_dim = 16
    learning_rate = 0.01
    epochs = 200
    target_class = 1  # The class label to target in the backdoor attack

    # Define models to evaluate
    models = {'GCN': GCN, 'GraphSAGE': GraphSAGE, 'GAT': GAT}

    # Store all results for final comparison table
    all_results = []

    # Run experiments for each dataset
    for dataset_name in datasets:
        print(f"Running experiment on dataset: {dataset_name}")
        dataset = load_dataset(dataset_name)
        data = dataset[0]

        # Create train, validation, and test masks if they do not exist
        if not hasattr(data, 'train_mask'):
            num_nodes = data.num_nodes
            train_ratio, val_ratio = 0.1, 0.1
            train_size = int(train_ratio * num_nodes)
            val_size = int(val_ratio * num_nodes)
            test_size = num_nodes - train_size - val_size

            perm = torch.randperm(num_nodes)
            data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
            data.val_mask = torch.zeros(num_nodes, dtype=torch.bool)
            data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)

            data.train_mask[perm[:train_size]] = True
            data.val_mask[perm[train_size:train_size + val_size]] = True
            data.test_mask[perm[train_size + val_size:]] = True
        input_dim = dataset.num_node_features
        output_dim = dataset.num_classes
        num_poisoned = attack_budgets[dataset_name]

        # Initialize trigger generator and OOD detector
        trigger_generator = TriggerGenerator(input_dim, hidden_dim)
        ood_detector = OODDetector(input_dim, hidden_dim)

        # Run experiments for each model
        for model_name, ModelClass in models.items():
            print(f"Running experiment for {model_name}")
            model = ModelClass(input_dim, hidden_dim, output_dim)
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)

            # Train on clean data
            for epoch in range(epochs):
                loss = train(model, data, optimizer)
                if epoch % 10 == 0:
                    print(f"Epoch {epoch}, Loss: {loss:.4f}")

            # Test on clean data
            clean_accs = test(model, data)
            print(f"{model_name} Clean Data Accuracy: Train: {clean_accs[0]:.4f}, Val: {clean_accs[1]:.4f}, Test: {clean_accs[2]:.4f}")

            # Add backdoor trigger with adversarial training
            poisoned_data, poisoned_indices = add_backdoor(data.clone(), trigger_generator, ood_detector, target_class, num_poisoned)

            # Retrain on poisoned data (No Defense)
            for epoch in range(epochs):
                loss = train(model, poisoned_data, optimizer)
                if epoch % 10 == 0:
                    print(f"Epoch {epoch} (Poisoned), Loss: {loss:.4f}")

            # Test on poisoned data (No Defense)
            poisoned_accs = test(model, poisoned_data)
            poisoned_preds = model(poisoned_data).argmax(dim=1)
            asr = (poisoned_preds[poisoned_indices] == target_class).float().mean().item()
            print(f"{model_name} No Defense - Poisoned Data Accuracy: Train: {poisoned_accs[0]:.4f}, Val: {poisoned_accs[1]:.4f}, Test: {poisoned_accs[2]:.4f}, ASR: {asr:.4f}")

            # Apply outlier detection defense
            defended_data, mask_indices = apply_outlier_detection(poisoned_data.clone(), ood_detector)
            valid_poisoned_indices = [mask_indices.tolist().index(i) for i in poisoned_indices if i in mask_indices]

            # Retrain on defended data
            for epoch in range(epochs):
                loss = train(model, defended_data, optimizer)
                if epoch % 10 == 0:
                    print(f"Epoch {epoch} (Defended), Loss: {loss:.4f}")

            # Test on defended data
            defended_accs = test(model, defended_data)
            defended_preds = model(defended_data).argmax(dim=1)
            defended_asr = (defended_preds[valid_poisoned_indices] == target_class).float().mean().item() if len(valid_poisoned_indices) > 0 else 0.0
            print(f"{model_name} Outlier Detection - Defended Data Accuracy: Train: {defended_accs[0]:.4f}, Val: {defended_accs[1]:.4f}, Test: {defended_accs[2]:.4f}, ASR: {defended_asr:.4f}")

            # Store results for comparison
            all_results.append({
                'Dataset': dataset_name,
                'Model': model_name,
                'Clean Test Accuracy': clean_accs[2],
                'Poisoned Test Accuracy (No Defense)': poisoned_accs[2],
                'Defended Test Accuracy (Outlier Detection)': defended_accs[2],
                'Attack Success Rate (No Defense)': asr,
                'Attack Success Rate (Outlier Detection)': defended_asr
            })

    # Create a DataFrame for the results
    results_df = pd.DataFrame(all_results)
    print("Final Comparison Table:")
    print(results_df)

    # Save the results to an Excel file
    results_df.to_excel('experiment_results.xlsx', index=False)

    # Plot the results
    fig, axes = plt.subplots(2, 1, figsize=(12, 12))

    # Plot Clean, Poisoned, and Defended Accuracies
    for dataset_name in datasets:
        subset = results_df[results_df['Dataset'] == dataset_name]
        axes[0].plot(subset['Model'], subset['Clean Test Accuracy'], marker='o', label=f'{dataset_name} - Clean')
        axes[0].plot(subset['Model'], subset['Poisoned Test Accuracy (No Defense)'], marker='x', linestyle='--', label=f'{dataset_name} - Poisoned (No Defense)')
        axes[0].plot(subset['Model'], subset['Defended Test Accuracy (Outlier Detection)'], marker='^', linestyle='-.', label=f'{dataset_name} - Defended')
    axes[0].set_title('Clean, Poisoned, and Defended Test Accuracies')
    axes[0].set_xlabel('Model')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True)

    # Plot Attack Success Rates
    for dataset_name in datasets:
        subset = results_df[results_df['Dataset'] == dataset_name]
        axes[1].plot(subset['Model'], subset['Attack Success Rate (No Defense)'], marker='o', linestyle='-', label=f'{dataset_name} - ASR (No Defense)')
        axes[1].plot(subset['Model'], subset['Attack Success Rate (Outlier Detection)'], marker='x', linestyle='--', label=f'{dataset_name} - ASR (Outlier Detection)')
    axes[1].set_title('Attack Success Rates')
    axes[1].set_xlabel('Model')
    axes[1].set_ylabel('Attack Success Rate')
    axes[1].legend()
    axes[1].grid(True)

    # Show the plots
    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    run_experiment()

Running experiment on dataset: Cora
Running experiment for GCN
Epoch 0, Loss: 1.9414
Epoch 10, Loss: 0.6484
Epoch 20, Loss: 0.2111
Epoch 30, Loss: 0.0893
Epoch 40, Loss: 0.0685
Epoch 50, Loss: 0.0302
Epoch 60, Loss: 0.0195
Epoch 70, Loss: 0.0253
Epoch 80, Loss: 0.0153
Epoch 90, Loss: 0.0140
Epoch 100, Loss: 0.0105
Epoch 110, Loss: 0.0111
Epoch 120, Loss: 0.0050
Epoch 130, Loss: 0.0091
Epoch 140, Loss: 0.0085
Epoch 150, Loss: 0.0110
Epoch 160, Loss: 0.0158
Epoch 170, Loss: 0.0207
Epoch 180, Loss: 0.0102
Epoch 190, Loss: 0.0078
GCN Clean Data Accuracy: Train: 1.0000, Val: 0.7440, Test: 0.7750
Adversarial Epoch 0, OOD Loss: 1.3888, Generator Loss: 1.7817
Adversarial Epoch 10, OOD Loss: 1.3275, Generator Loss: 0.9446
Adversarial Epoch 20, OOD Loss: 0.4477, Generator Loss: 9.6050
Adversarial Epoch 30, OOD Loss: 0.1584, Generator Loss: 9.3286
Adversarial Epoch 40, OOD Loss: 0.1510, Generator Loss: 13.3119
Epoch 0 (Poisoned), Loss: 0.0106
Epoch 10 (Poisoned), Loss: 0.0036
Epoch 20 (Poisoned),

/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Epoch 0, Loss: 1.0940
Epoch 10, Loss: 0.6840
Epoch 20, Loss: 0.5111
Epoch 30, Loss: 0.3855
Epoch 40, Loss: 0.4200
Epoch 50, Loss: 0.2304
Epoch 60, Loss: 0.2168
Epoch 70, Loss: 0.3345
Epoch 80, Loss: 0.1779
Epoch 90, Loss: 0.2423
Epoch 100, Loss: 0.2420
Epoch 110, Loss: 0.3326
Epoch 120, Loss: 0.2048
Epoch 130, Loss: 0.1984
Epoch 140, Loss: 0.2583
Epoch 150, Loss: 0.2142
Epoch 160, Loss: 0.2089
Epoch 170, Loss: 0.2141
Epoch 180, Loss: 0.2038
Epoch 190, Loss: 0.2221
GAT Clean Data Accuracy: Train: 1.0000, Val: 0.7580, Test: 0.7580
Adversarial Epoch 0, OOD Loss: 0.6100, Generator Loss: 3.6418
Adversarial Epoch 10, OOD Loss: 0.5126, Generator Loss: 3.2682
Adversarial Epoch 20, OOD Loss: 0.6106, Generator Loss: 2.3010
Adversarial Epoch 30, OOD Loss: 0.4740, Generator Loss: 3.4712
Adversarial Epoch 40, OOD Loss: 0.5138, Generator Loss: 2.8087
Epoch 0 (Poisoned), Loss: 0.2525
Epoch 10 (Poisoned), Loss: 0.2309
Epoch 20 (Poisoned), Loss: 0.2175
Epoch 30 (Poisoned), Loss: 0.1484
Epoch 40 (Poison

/opt/conda/lib/python3.10/site-packages/ogb/nodeproppred/dataset_pyg.py:69: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.data, self.slices = torch.load(self.processed_

RuntimeError: 0D or 1D target tensor expected, multi-target not supported